In [1]:
#Grid LG boost regressor
import pandas as pd
import lightgbm

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [8]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [10]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
param_grid={"boosting_type":["gbdt","str","dart","rf"],
            "estimators":[100,200,500,1000],
            "class_weight":["balanced"]}
grid= GridSearchCV(LGBMRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[LightGBM] [Warning] Unknown parameter: estimators


C:\Users\ARUN\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ARUN\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ARUN\Anaconda3\lib\site-packages\lightgbm\sklearn.py", line 822, in fit
    categorical_feature=categorical_feature, callbacks=callbacks, init_model=init_model)
  File "C:\Users\ARUN\Anaconda3\lib\site-packages\lightgbm\sklearn.py", line 688, in fit
    callbacks=callbacks, init_model=init_mode

GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt', 'str', 'dart', 'rf'],
                         'class_weight': ['balanced'],
                         'estimators': [100, 200, 500, 1000]},
             verbose=3)

In [11]:
re=grid.cv_results_

In [12]:
y_pred=grid.predict(x_test)
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.86603193419773

In [13]:
print("The r_score value from parameter{}:".format(grid.best_params_),
      "The r_score value:",r_score)

The r_score value from parameter{'boosting_type': 'gbdt', 'class_weight': 'balanced', 'estimators': 100}: The r_score value: 0.86603193419773


In [14]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting_type,param_class_weight,param_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.143164,0.084443,0.010742,1.069597e-03,gbdt,balanced,100,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",0.861639,0.784171,0.822872,0.818837,0.780870,0.813678,0.029524,1
1,0.075977,0.004840,0.009765,8.996946e-07,gbdt,balanced,200,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",0.861639,0.784171,0.822872,0.818837,0.780870,0.813678,0.029524,1
2,0.074609,0.002278,0.010742,2.048546e-03,gbdt,balanced,500,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",0.861639,0.784171,0.822872,0.818837,0.780870,0.813678,0.029524,1
3,0.072460,0.002858,0.009570,3.914598e-04,gbdt,balanced,1000,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",0.861639,0.784171,0.822872,0.818837,0.780870,0.813678,0.029524,1
4,0.051759,0.045319,0.000000,0.000000e+00,str,balanced,100,"{'boosting_type': 'str', 'class_weight': 'bala...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
5,0.014260,0.003634,0.000000,0.000000e+00,str,balanced,200,"{'boosting_type': 'str', 'class_weight': 'bala...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
6,0.015821,0.005607,0.000000,0.000000e+00,str,balanced,500,"{'boosting_type': 'str', 'class_weight': 'bala...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
7,0.010155,0.000478,0.000000,0.000000e+00,str,balanced,1000,"{'boosting_type': 'str', 'class_weight': 'bala...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
8,0.106053,0.000995,0.008790,2.861023e-07,dart,balanced,100,"{'boosting_type': 'dart', 'class_weight': 'bal...",0.863888,0.787605,0.811774,0.831384,0.759772,0.810885,0.035735,5
9,0.108983,0.005580,0.008985,3.914601e-04,dart,balanced,200,"{'boosting_type': 'dart', 'class_weight': 'bal...",0.863888,0.787605,0.811774,0.831384,0.759772,0.810885,0.035735,5


In [15]:
age=int(input("AGE:"))
bmi=float(input("BMI:"))
childern=int(input("CHILDERN:"))
sex=int(input("MALE 1 or FEMALE 0:"))
smoker=int(input("SMOKER 1 or NON-SMOKER 0:"))




AGE:23
BMI:20
CHILDERN:1
MALE 1 or FEMALE 0:1
SMOKER 1 or NON-SMOKER 0:0


In [16]:
Future_prediction=grid.predict([[age,bmi,childern,sex,smoker]])
print("Future_prediction{}:".format(Future_prediction))

Future_prediction[4597.20747635]:


In [17]:
r_score

0.86603193419773